In [ ]:
import torch
import os
import json
os.getcwd()
os.chdir('/storage/etaisella/repos/shape_proj/')

from shap_e.models.download import load_model
from shap_e.util.data_util import load_or_create_multimodal_batch
from shap_e.util.notebooks import create_pan_cameras, decode_latent_images, gif_widget
import objaverse
import random
objaverse.__version__
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
xm = load_model('transmitter', device=device)
%env BLENDER_PATH=/storage/etaisella/blender-3.4.1-linux-x64/blender

### Get random objaverse model

In [ ]:
# setup dataset path
dataset_path = '/storage/etaisella/repos/shape_proj/objaverse_dataset'

# get model
#random.seed(42)
uids = objaverse.load_uids()
random_object_uids = random.sample(uids, 1)
objects = objaverse.load_objects(uids=random_object_uids)
annotations = objaverse.load_annotations(random_object_uids)
obj_id = random_object_uids[0]
obj_annotation = annotations[obj_id]
obj_name = obj_annotation['name']
obj_localpath = objects[obj_id]
print(f'id: {obj_id}, name: {obj_name}')

# make folders for model
subfolder_name = obj_name.replace(' ', '_') + f'_{obj_id}'
os.makedirs(os.path.join(dataset_path, subfolder_name), exist_ok=True)

# save annotation dict as json
with open(os.path.join(dataset_path, subfolder_name, 'annotation.json'), 'w') as f:
    json.dump(obj_annotation, f)

In [ ]:
model_path = objects[random_object_uids[0]]

# This may take a few minutes, since it requires rendering the model twice
# in two different modes.
batch = load_or_create_multimodal_batch(
    device,
    model_path=model_path,
    mv_light_mode="basic",
    mv_image_size=256,
    cache_dir="example_data/objaverse/cached",
    random_sample_count=2**17,
    verbose=True, # this will show Blender output during renders
)

In [ ]:
with torch.no_grad():
    latent = xm.encoder.encode_to_bottleneck(batch)

    render_mode = 'stf' # you can change this to 'nerf'
    size = 128 # recommended that you lower resolution when using nerf

    cameras = create_pan_cameras(size, device)
    images = decode_latent_images(xm, latent, cameras, rendering_mode=render_mode)

# save images
os.makedirs(os.path.join("example_data/objaverse", 'test_imgs'), exist_ok=True)
for i, img in enumerate(images):
    img.save(os.path.join("example_data/objaverse", 'test_imgs', f'{i:05}.png'))

### Get annotation

In [ ]:
annotations = objaverse.load_annotations(random_object_uids)
print(annotations)